In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [2]:
def retrieval_agent(fetch_k:int, k:int):
    embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="bge-m3:latest")
    vectorstore = Chroma(collection_name="collection_01", persist_directory="./db/chroma_db_02", embedding_function=embed_model)
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': k, "fetch_k":fetch_k})
    return retriever

retriever = retrieval_agent(fetch_k=10, k=3)
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001E13F49EF30>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10})

In [3]:
question = " according to lr rule, explain about ships with installed process plant for chemicals"
docs = retriever.invoke(question)
docs

[Document(metadata={'File Name': "Lloyd's Register Rules and Regulations for the Classification of Ships, July 2022", 'File Path': "/content/drive/MyDrive/Rules/LR/Lloyd's Register Rules and Regulations for the Classification of Ships, July 2022.pdf", 'First Division': 'Rules', 'Page': 1650, 'Second Division': 'LR'}, page_content="This page explains Lloyd's Register Rules and Regulations for the Classification of Ships, July 2022, that belongs to catogories of Rules and LR./nwith the process plant are to be submitted, including: • Arrangements for preventing the ingress of water into the ship or structure where process plant and equipment protrude through the weather deck. • Proposed emergency flooding procedures and their control. 3.2.6 Particulars of the proposed storage arrangements of hazardous and/or toxic substances, feedstocks and products in bulk, on the ship or structure, are to be submitted. 3.3 Process plant 3.3.1 A description of the expected method of operation of the proc

In [4]:
def generate_agent(question:str, context:list):
    prompt = ChatPromptTemplate.from_messages([
    ("human", 
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:"""),
    ])
    llm = ChatGroq(temperature=0, model_name= "llama-3.3-70b-versatile")

    rag_chain = prompt | llm | StrOutputParser()

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    total_docs = format_docs(context)

    generation = rag_chain.invoke({"context": total_docs, "question": question})
    return generation

generation = generate_agent(question=question, context=docs)
generation

"According to the Lloyd's Register (LR) rules, ships with installed process plants for chemicals must submit various details, including arrangements for preventing water ingress and emergency flooding procedures. The rules also require a description of the process plant's method of operation and a diagram showing the process flow. Additionally, ships with process plants may be assigned special notations, such as a chemical cargoes notation, depending on their category and operation."

In [5]:
from pprint import pprint
pprint(generation)

("According to the Lloyd's Register (LR) rules, ships with installed process "
 'plants for chemicals must submit various details, including arrangements for '
 'preventing water ingress and emergency flooding procedures. The rules also '
 "require a description of the process plant's method of operation and a "
 'diagram showing the process flow. Additionally, ships with process plants '
 'may be assigned special notations, such as a chemical cargoes notation, '
 'depending on their category and operation.')
